In [1]:
#############################################################################
# zlib License
#
# (C) 2023 Zach Flowers, Murtaza Safdari <musafdar@cern.ch>
#
# This software is provided 'as-is', without any express or implied
# warranty.  In no event will the authors be held liable for any damages
# arising from the use of this software.
#
# Permission is granted to anyone to use this software for any purpose,
# including commercial applications, and to alter it and redistribute it
# freely, subject to the following restrictions:
#
# 1. The origin of this software must not be misrepresented; you must not
#    claim that you wrote the original software. If you use this software
#    in a product, an acknowledgment in the product documentation would be
#    appreciated but is not required.
# 2. Altered source versions must be plainly marked as such, and must not be
#    misrepresented as being the original software.
# 3. This notice may not be removed or altered from any source distribution.
#############################################################################

# Imports

In [2]:
#%%
%matplotlib inline
import matplotlib.pyplot as plt
import logging
import i2c_gui
import i2c_gui.chips
from i2c_gui.usb_iss_helper import USB_ISS_Helper
from i2c_gui.fpga_eth_helper import FPGA_ETH_Helper
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
# import time
from tqdm import tqdm
# from i2c_gui.chips.etroc2_chip import register_decoding
import os, sys
import multiprocessing
os.chdir(f'/home/{os.getlogin()}/ETROC2/ETROC_DAQ')
import run_script
import parser_arguments
import importlib
importlib.reload(run_script)
import datetime
import pandas
from pathlib import Path
import subprocess
import sqlite3
from notebooks.notebook_helpers import *
from fnmatch import fnmatch
import scipy.stats as stats
from math import ceil
from numpy import savetxt

# Set defaults

In [3]:
# !!!!!!!!!!!!
# It is very important to correctly set the chip name, this value is stored with the data
chip_names = ["ET2_EPIR_Pair1", "ET2_BAR_4", "ET2_BAR_6", "CNM_HPK_1_3"]
chip_fignames = ["EPIR Pair 1", "Barcelona 4", "Barcelona 6", "CNM (HPK) 1-3"]
# chip_names = ["ET2_EPIR_Pair4"]
# chip_fignames = ["EPIR Pair 4"]
chip_figtitles = chip_names

# 'The port name the USB-ISS module is connected to. Default: /dev/ttyACM0'
port = "/dev/ttyACM0"
# I2C addresses for the pixel block and WS
chip_addresses = [0x60, 0x61, 0x62, 0x64]
# chip_addresses = [0x63]
ws_addresses = [None, None, None, None]

fig_outdir = Path('../ETROC-figures')
fig_outdir = fig_outdir / (datetime.date.today().isoformat() + '_Array_Test_Results')
fig_outdir.mkdir(exist_ok=True)
fig_path = str(fig_outdir)

# Make i2c_connection class object

In [4]:
# i2c_conn = self, port, chip_addresses, chip_names, chip_fc_delays
i2c_conn = i2c_connection(port,chip_addresses,ws_addresses,chip_names,[("1","1"),("1","1"),("1","1"), ("1","1")])

# Config chips

### Key is (Disable Pixels, Auto Cal, Chip Peripherals, Basic Peri Reg Check, Pixel Check)

In [8]:
# (WS Prep Pixel and Peri) - 0 - (disable & auto_cal all pixels) - (disable default all pixels) - (auto_TH_CAL) - (set basic peripherals) - (peripheral reg check) -  (pixel ID check)
i2c_conn.config_chips('00100111')
# i2c_conn.config_chips('00001111') # disable and more fun things
# i2c_conn.config_chips('00000100') # set basic peripherals
# i2c_conn.config_chips('00010000') # calibrate only
# i2c_conn.config_chips('00000001') # pixel ID check (I2C check)
# i2c_conn.config_chips('10000000') # ws init
# i2c_conn.disable_all_pixels(chip_address=chip_addresses[1])

Passed pixel check for chip: 0x60
Passed peripheral write check for chip: 0x60
Peripherals set for chip: 0x60
Disabled pixels for chip: 0x60


Calibrating row: 100%|██████████| 16/16 [01:28<00:00,  5.50s/it]


Auto calibration finished for chip: 0x60
Passed pixel check for chip: 0x61
Passed peripheral write check for chip: 0x61
Peripherals set for chip: 0x61
Disabled pixels for chip: 0x61


Calibrating row: 100%|██████████| 16/16 [01:27<00:00,  5.49s/it]


Auto calibration finished for chip: 0x61
Passed pixel check for chip: 0x62
Passed peripheral write check for chip: 0x62
Peripherals set for chip: 0x62
Disabled pixels for chip: 0x62


Calibrating row: 100%|██████████| 16/16 [01:27<00:00,  5.49s/it]


Auto calibration finished for chip: 0x62
Passed pixel check for chip: 0x64
Passed peripheral write check for chip: 0x64
Peripherals set for chip: 0x64
Disabled pixels for chip: 0x64


Calibrating row: 100%|██████████| 16/16 [01:27<00:00,  5.49s/it]

Auto calibration finished for chip: 0x64


## Visualize the learned Baselines (BL) and Noise Widths (NW)

Note that the NW represents the full width on either side of the BL

In [5]:
#%%
%matplotlib inline
import matplotlib.pyplot as plt
plt.figure()
plt.show()

<Figure size 1000x1000 with 0 Axes>

In [10]:
histdir = Path('/run/media/daq/T7/ETROC-History-DESY')
histdir.mkdir(exist_ok=True)
histfile = histdir / 'BaselineHistory.sqlite'

# from mpl_toolkits.axes_grid1 import make_axes_locatable
for chip_address, chip_figname, chip_figtitle in zip(chip_addresses,chip_fignames,chip_fignames):
    BL_map_THCal,NW_map_THCal,BL_df = i2c_conn.get_auto_cal_maps(chip_address)
    fig = plt.figure(dpi=200, figsize=(20,10))
    gs = fig.add_gridspec(1,2)

    ax0 = fig.add_subplot(gs[0,0])
    ax0.set_title(f"{chip_figtitle}: BL (DAC LSB)", size=17, loc="right")
    img0 = ax0.imshow(BL_map_THCal, interpolation='none')
    ax0.set_aspect("equal")
    ax0.invert_xaxis()
    ax0.invert_yaxis()
    plt.xticks(range(16), range(16), rotation="vertical")
    plt.yticks(range(16), range(16))
    hep.cms.text(loc=0, ax=ax0, fontsize=17, text="Preliminary")
    divider = make_axes_locatable(ax0)
    cax = divider.append_axes('right', size="5%", pad=0.05)
    fig.colorbar(img0, cax=cax, orientation="vertical")

    ax1 = fig.add_subplot(gs[0,1])
    ax1.set_title(f"{chip_figtitle}: NW (DAC LSB)", size=17, loc="right")
    img1 = ax1.imshow(NW_map_THCal, interpolation='none')
    ax1.set_aspect("equal")
    ax1.invert_xaxis()
    ax1.invert_yaxis()
    plt.xticks(range(16), range(16), rotation="vertical")
    plt.yticks(range(16), range(16))
    hep.cms.text(loc=0, ax=ax1, fontsize=17, text="Preliminary")
    divider = make_axes_locatable(ax1)
    cax = divider.append_axes('right', size="5%", pad=0.05)
    fig.colorbar(img1, cax=cax, orientation="vertical")

    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")

    for x in range(16):
        for y in range(16):
            ax0.text(x,y,f"{BL_map_THCal.T[x,y]:.0f}", c="white", size=10, rotation=45, fontweight="bold", ha="center", va="center")
            ax1.text(x,y,f"{NW_map_THCal.T[x,y]:.0f}", c="white", size=10, rotation=45, fontweight="bold", ha="center", va="center")
    plt.tight_layout()
    plt.savefig(fig_path+"/BL_NW_"+chip_figname+"_"+timestamp+".png")
    plt.show()

    with sqlite3.connect(histfile) as sqlconn:
        BL_df.to_sql('baselines', sqlconn, if_exists='append', index=False)

    savetxt(histdir / f'{chip_figname}_BL_{timestamp}.csv', BL_map_THCal, delimiter=',')
    savetxt(histdir / f'{chip_figname}_NW_{timestamp}.csv', NW_map_THCal, delimiter=',')

In [11]:
for chip_address, chip_name in zip(chip_addresses, chip_names):
    i2c_conn.save_auto_cal_BL_map(chip_address, chip_name, "")
    i2c_conn.save_auto_cal_NW_map(chip_address, chip_name, "")

In [ ]:
for chip_address, chip_name in zip(chip_addresses, chip_names):
    i2c_conn.load_auto_cal_BL_map(chip_address, chip_name, "")
    i2c_conn.load_auto_cal_NW_map(chip_address, chip_name, "")

# Define pixels of interest

In [12]:
# row_list = [15, 15, 15, 15]
# col_list = [6, 7, 8, 9]
# row_list = [14, 14, 14, 14]
# col_list = [6, 7, 8, 9]
# row_list = [9, 3, 12]
# col_list = [3, 8, 11]
# row_list = [15, 15]
# col_list = [6, 9]
# row_list = [14, 14, 15, 15]
# col_list = [6, 9, 6, 9]
# row_list = [14, 14, 14, 14, 15, 15, 15, 15]
# col_list = [6, 7, 8, 9, 6, 7, 8, 9]
# col_list = np.arange(16)
# row_list = np.full_like(col_list,12)
# row_list = np.full_like(col_list,13)
# col_list = [10, 13, 7, 4, 10, 13, 7, 4]
# row_list = [12, 12, 12, 12, 13, 13, 13, 13]

# scan_list = list(zip(row_list, col_list))

col_list, row_list = np.meshgrid(np.arange(16),np.arange(16))
scan_list = list(zip(row_list.flatten(),col_list.flatten()))

print(scan_list)

[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (0, 10), (0, 11), (0, 12), (0, 13), (0, 14), (0, 15), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (1, 11), (1, 12), (1, 13), (1, 14), (1, 15), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (2, 11), (2, 12), (2, 13), (2, 14), (2, 15), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (3, 10), (3, 11), (3, 12), (3, 13), (3, 14), (3, 15), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (4, 10), (4, 11), (4, 12), (4, 13), (4, 14), (4, 15), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6), (5, 7), (5, 8), (5, 9), (5, 10), (5, 11), (5, 12), (5, 13), (5, 14), (5, 15), (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 6), (6, 7), (6, 8), (6, 9), (6, 10), (6, 11), (6, 12), (6, 13), (6, 14), (6, 15), (7, 0), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 7

In [13]:
for chip_address in chip_addresses[:]:
    chip = i2c_conn.get_chip_i2c_connection(chip_address)
    row_indexer_handle,_,_ = chip.get_indexer("row")
    column_indexer_handle,_,_ = chip.get_indexer("column")
    IBSel_handle = chip.get_decoded_indexed_var("ETROC2", "Pixel Config", "IBSel")
    for row, col in scan_list:
        print(f"Changing to high power chip {hex(chip_address)} Pixel ({row},{col})")
        column_indexer_handle.set(col)
        row_indexer_handle.set(row)
        chip.read_all_block("ETROC2", "Pixel Config")
        IBSel_handle.set("000")
        chip.write_all_block("ETROC2", "Pixel Config")
    del chip, row_indexer_handle, column_indexer_handle


Changing to high power chip 0x60 Pixel (0,0)
Changing to high power chip 0x60 Pixel (0,1)
Changing to high power chip 0x60 Pixel (0,2)
Changing to high power chip 0x60 Pixel (0,3)
Changing to high power chip 0x60 Pixel (0,4)
Changing to high power chip 0x60 Pixel (0,5)
Changing to high power chip 0x60 Pixel (0,6)
Changing to high power chip 0x60 Pixel (0,7)
Changing to high power chip 0x60 Pixel (0,8)
Changing to high power chip 0x60 Pixel (0,9)
Changing to high power chip 0x60 Pixel (0,10)
Changing to high power chip 0x60 Pixel (0,11)
Changing to high power chip 0x60 Pixel (0,12)
Changing to high power chip 0x60 Pixel (0,13)
Changing to high power chip 0x60 Pixel (0,14)
Changing to high power chip 0x60 Pixel (0,15)
Changing to high power chip 0x60 Pixel (1,0)
Changing to high power chip 0x60 Pixel (1,1)
Changing to high power chip 0x60 Pixel (1,2)
Changing to high power chip 0x60 Pixel (1,3)
Changing to high power chip 0x60 Pixel (1,4)
Changing to high power chip 0x60 Pixel (1,5)
Chan

### Enable pixels of Interest

In [14]:
i2c_conn.enable_select_pixels_in_chips(scan_list,QInjEn=False,Bypass_THCal=False,specified_addresses=chip_addresses[:],power_mode='high')

Enabled pixel (0,0) for chip: 0x60
Enabled pixel (0,1) for chip: 0x60
Enabled pixel (0,2) for chip: 0x60
Enabled pixel (0,3) for chip: 0x60
Enabled pixel (0,4) for chip: 0x60
Enabled pixel (0,5) for chip: 0x60
Enabled pixel (0,6) for chip: 0x60
Enabled pixel (0,7) for chip: 0x60
Enabled pixel (0,8) for chip: 0x60
Enabled pixel (0,9) for chip: 0x60
Enabled pixel (0,10) for chip: 0x60
Enabled pixel (0,11) for chip: 0x60
Enabled pixel (0,12) for chip: 0x60
Enabled pixel (0,13) for chip: 0x60
Enabled pixel (0,14) for chip: 0x60
Enabled pixel (0,15) for chip: 0x60
Enabled pixel (1,0) for chip: 0x60
Enabled pixel (1,1) for chip: 0x60
Enabled pixel (1,2) for chip: 0x60
Enabled pixel (1,3) for chip: 0x60
Enabled pixel (1,4) for chip: 0x60
Enabled pixel (1,5) for chip: 0x60
Enabled pixel (1,6) for chip: 0x60
Enabled pixel (1,7) for chip: 0x60
Enabled pixel (1,8) for chip: 0x60
Enabled pixel (1,9) for chip: 0x60
Enabled pixel (1,10) for chip: 0x60
Enabled pixel (1,11) for chip: 0x60
Enabled pixe

In [15]:
offset = 0x0f
for chip_address in chip_addresses[:]:
    chip = i2c_conn.get_chip_i2c_connection(chip_address)
    row_indexer_handle,_,_ = chip.get_indexer("row")
    column_indexer_handle,_,_ = chip.get_indexer("column")
    for row, col in scan_list:
        print(f"Operating on chip {hex(chip_address)} Pixel ({row},{col})")
        column_indexer_handle.set(col)
        row_indexer_handle.set(row)
        # i2c_conn.pixel_decoded_register_write("QSel", format(0x14, '05b'), chip)
        i2c_conn.pixel_decoded_register_write("TH_offset", format(offset, '06b'), chip)
    del chip, row_indexer_handle, column_indexer_handle

Operating on chip 0x60 Pixel (0,0)
Operating on chip 0x60 Pixel (0,1)
Operating on chip 0x60 Pixel (0,2)
Operating on chip 0x60 Pixel (0,3)
Operating on chip 0x60 Pixel (0,4)
Operating on chip 0x60 Pixel (0,5)
Operating on chip 0x60 Pixel (0,6)
Operating on chip 0x60 Pixel (0,7)
Operating on chip 0x60 Pixel (0,8)
Operating on chip 0x60 Pixel (0,9)
Operating on chip 0x60 Pixel (0,10)
Operating on chip 0x60 Pixel (0,11)
Operating on chip 0x60 Pixel (0,12)
Operating on chip 0x60 Pixel (0,13)
Operating on chip 0x60 Pixel (0,14)
Operating on chip 0x60 Pixel (0,15)
Operating on chip 0x60 Pixel (1,0)
Operating on chip 0x60 Pixel (1,1)
Operating on chip 0x60 Pixel (1,2)
Operating on chip 0x60 Pixel (1,3)
Operating on chip 0x60 Pixel (1,4)
Operating on chip 0x60 Pixel (1,5)
Operating on chip 0x60 Pixel (1,6)
Operating on chip 0x60 Pixel (1,7)
Operating on chip 0x60 Pixel (1,8)
Operating on chip 0x60 Pixel (1,9)
Operating on chip 0x60 Pixel (1,10)
Operating on chip 0x60 Pixel (1,11)
Operating on

### Individual board offsets

In [ ]:
board_offsets = {
    0x60: 0x0f,
    0x61: 0x08,
    0x62: 0x0f,
    0x64: 0x08,
}

for chip_address in chip_addresses[:]:
    if chip_address not in board_offsets:
        continue

    chip = i2c_conn.get_chip_i2c_connection(chip_address)
    row_indexer_handle,_,_ = chip.get_indexer("row")
    column_indexer_handle,_,_ = chip.get_indexer("column")
    for row, col in scan_list:
        print(f"Operating on chip {hex(chip_address)} Pixel ({row},{col})")
        column_indexer_handle.set(col)
        row_indexer_handle.set(row)
        # i2c_conn.pixel_decoded_register_write("QSel", format(0x14, '05b'), chip)
        i2c_conn.pixel_decoded_register_write("TH_offset", format(board_offsets[chip_address], '06b'), chip)
    del chip, row_indexer_handle, column_indexer_handle

### QInj Pixels

In [ ]:
qinj_pixels = {
    0x60: [ # (row, col)
        (6, 6),
    ],
    0x61: [ # (row, col)
        (7, 7),
    ],
    0x62: [ # (row, col)
        (8, 8),
    ],
    0x63: [ # (row, col)
        (9, 9),
    ],
}

for chip_address in chip_addresses:
    if chip_address not in qinj_pixels:
        continue
    chip = i2c_conn.get_chip_i2c_connection(chip_address)
    row_indexer_handle,_,_ = chip.get_indexer("row")
    column_indexer_handle,_,_ = chip.get_indexer("column")
    for row, col in qinj_pixels[chip_address]:
        print(f"Enabling QInj of chip {hex(chip_address)} Pixel ({row},{col})")
        column_indexer_handle.set(col)
        row_indexer_handle.set(row)
        # i2c_conn.pixel_decoded_register_write("QSel", format(0x14, '05b'), chip)
        i2c_conn.pixel_decoded_register_write("QInjEn", '1', chip)


### Offset Pixel per board

In [ ]:
offset_pixels = {
    0x60: { # (row, col): offset
        (10, 8): 15,
    }
}

for chip_address in chip_addresses:
    if chip_address not in offset_pixels:
        continue
    chip = i2c_conn.get_chip_i2c_connection(chip_address)
    row_indexer_handle,_,_ = chip.get_indexer("row")
    column_indexer_handle,_,_ = chip.get_indexer("column")
    for row, col in offset_pixels[chip_address]:
        print(f"Setting offset of chip {hex(chip_address)} Pixel ({row},{col})")
        column_indexer_handle.set(col)
        row_indexer_handle.set(row)
        i2c_conn.pixel_decoded_register_write("TH_offset", format(offset_pixels[chip_address][(row, col)], '06b'), chip)

### Remove noisy pixels from Trigger

In [16]:
# noisy_pixels = {
#     0x60: [ # (row, col)
#         (10, 8),
#         (13, 4),
#         (1, 3)
#     ],
#     0x61: [ # (row, col)
#         (15, 0),
#         (1, 15),
#         (2, 9),
#     ],
#     0x62: [ # (row, col)
#         (14, 2),
#     ],
#     0x63: [ # (row, col)
#         (13, 2),
#         (7, 8),
#     ],
# }

noisy_pixels = {
    0x60: [ # (row, col)
        (10, 8),
        (13, 4),
    ],
}

for chip_address in chip_addresses:
    if chip_address not in noisy_pixels:
        continue
    chip = i2c_conn.get_chip_i2c_connection(chip_address)
    row_indexer_handle,_,_ = chip.get_indexer("row")
    column_indexer_handle,_,_ = chip.get_indexer("column")
    for row, col in noisy_pixels[chip_address]:
        print(f"Masking from trigger of chip {hex(chip_address)} Pixel ({row},{col})")
        column_indexer_handle.set(col)
        row_indexer_handle.set(row)

        # i2c_conn.enable_pixel_modular(row=row, col=col, verbose=True, chip_address=chip_address, chip=chip, row_indexer_handle=row_indexer_handle, column_indexer_handle=column_indexer_handle, QInjEn=False, Bypass_THCal=False, triggerWindow=True, cbWindow=True)
        # i2c_conn.pixel_decoded_register_write("TH_offset", format(offset, '06b'), chip)

        # i2c_conn.disable_pixel(row=row, col=col, verbose=True, chip_address=chip_address, chip=chip, row_indexer_handle=row_indexer_handle, column_indexer_handle=column_indexer_handle)
        i2c_conn.pixel_decoded_register_write("disTrigPath", "1", chip)
        #i2c_conn.pixel_decoded_register_write("DAC", format(1023, '010b'), chip)
        #i2c_conn.pixel_decoded_register_write("Bypass_THCal", "0", chip)



Masking from trigger of chip 0x60 Pixel (10,8)
Masking from trigger of chip 0x60 Pixel (13,4)


### Disable Pixels of Interest

In [ ]:
for chip_address in chip_addresses[:]:
    chip = i2c_conn.get_chip_i2c_connection(chip_address)
    row_indexer_handle,_,_ = chip.get_indexer("row")
    column_indexer_handle,_,_ = chip.get_indexer("column")
    for row,col in scan_list:
        i2c_conn.disable_pixel(row=row, col=col, verbose=True, chip_address=chip_address, chip=chip, row_indexer_handle=row_indexer_handle, column_indexer_handle=column_indexer_handle)
    del chip, row_indexer_handle, column_indexer_handle

# Calibrate FC for all I2C

In [18]:
for chip_address in chip_addresses[:]:
    i2c_conn.asyResetGlobalReadout(chip_address, chip=None)
    i2c_conn.asyAlignFastcommand(chip_address, chip=None)

Reset Global Readout done for chip: 0x60
asyAlignFastcommand action done for chip: 0x60
Reset Global Readout done for chip: 0x61
asyAlignFastcommand action done for chip: 0x61
Reset Global Readout done for chip: 0x62
asyAlignFastcommand action done for chip: 0x62
Reset Global Readout done for chip: 0x64
asyAlignFastcommand action done for chip: 0x64


# Calibrate PLL

In [ ]:
for chip_address in chip_addresses[:]:
    i2c_conn.calibratePLL(chip_address, chip=None)

# Disconnect I2C Device

## DO THIS IF YOU PLAN TO DO NOISE STUDIES

In [ ]:
del i2c_conn